📘 Example: Creating a Synthetic Event in FloodAdapt

This notebook demonstrates how to create a synthetic event using FloodAdapt. Synthetic events are valuable for controlled testing, sensitivity analysis, and understanding the behavior of flood models under simplified or hypothetical scenarios. Here, we construct a full synthetic event with water level, rainfall, wind, and river discharge forcings, and then save it to a FloodAdapt database.

⏱️ 1. Setup and Imports

We begin by importing the required classes and modules for constructing synthetic forcings and managing event data within the flood_adapt framework.

In [ ]:
import flood_adapt.objects.forcing as f

from pathlib import Path
from datetime import datetime

from flood_adapt.objects import SyntheticEvent, TimeFrame
from flood_adapt import unit_system as us
from flood_adapt import FloodAdapt
from flood_adapt.config.sfincs import RiverModel

🗓️ 2. Define the Simulation Time Frame

We specify a one-day time frame for the synthetic event, from January 1 to January 2, 2025.

In [ ]:
# Create an time frame for the simulation
start_time = datetime(year=2025, month=1, day=1)
end_time = datetime(year=2025, month=1, day=2)
time_frame = TimeFrame(start_time=start_time, end_time=end_time)

🌊 3. Define Water Level Forcing

Synthetic water levels are constructed from a combination of tidal and surge components.

In [ ]:
# Synthetic water levels are computed from a combination of surge and tide.
surge = f.SurgeModel(
    timeseries=f.SyntheticTimeseries(
        shape_type=f.ShapeType.gaussian,
        duration=us.UnitfulTime(value=12, units=us.UnitTypesTime.hours),
        peak_time=us.UnitfulTime.from_timedelta(time_frame.duration / 2), # Choose the middle of the time frame for peak time
        peak_value=us.UnitfulLength(value=2, units=us.UnitTypesLength.meters),
    )
)

tide = f.TideModel(
    harmonic_amplitude=us.UnitfulLength(value=1, units=us.UnitTypesLength.meters),
    harmonic_phase=us.UnitfulTime.from_timedelta(time_frame.duration / 2), # Choose the middle of the time frame for peak time
    harmonic_period=us.UnitfulTime(value=12.4, units=us.UnitTypesTime.hours),
)

water_levels = f.WaterlevelSynthetic(
    surge=surge,
    tide=tide,
)

# Inspect
df = water_levels.to_dataframe(time_frame=time_frame)
df.plot(title="Synthetic Water Levels", xlabel="Time", ylabel="Water Level (m)", legend=True, figsize=(5, 2))

🌧️ 4. Define Rainfall Forcing

We apply a constant rainfall intensity throughout the simulation period.

In [ ]:
# Choose constant rainfall
rainfall = f.RainfallConstant(
    intensity=us.UnitfulIntensity(value=10, units=us.UnitTypesIntensity.mm_hr)
)

# Inspect
df = rainfall.to_dataframe(time_frame=time_frame)
df.plot(title="Constant Rainfall", xlabel="Time", ylabel="Rainfall Intensity (mm/hr)", legend=True, figsize=(5, 2))

🌬️ 5. Define Wind Forcing

A constant wind speed and direction is specified to simulate wind-driven effects on the water surface.

In [ ]:
wind = f.WindConstant(
    speed=us.UnitfulVelocity(value=10, units=us.UnitTypesVelocity.mps), 
    direction=us.UnitfulDirection(value=0, units=us.UnitTypesDirection.degrees) # 0 degrees is North, 90 degrees is East, 180 degrees is South, and 270 degrees is West
)

# Inspect
df = wind.to_dataframe(time_frame=time_frame)
ax = df.plot(y='mag', ylabel='Wind Speed (m/s)', title='Constant Wind', legend=True, figsize=(5, 2))
df.plot(y='dir', secondary_y=True, ax=ax, ylabel='Wind Direction (degrees)', legend=True)

🏞️ 6. Define River Discharge Forcing

Discharge is defined for two pre-configured rivers. These rivers must be registered in the hazard model configuration beforehand.

In [ ]:
# Add discharge
# The available rivers are defined in the sfincs model when creating the database.
# You cannot add new rivers to the model in an event, you can only set the discharge of each given river.
river1 = RiverModel(
    name="River1",
    x_coordinate=1,
    y_coordinate=1,
    mean_discharge=us.UnitfulDischarge(value=100, units=us.UnitTypesDischarge.cms), 
    # Mean discharge is used create default discharge values for the river, you can still overwrite it later.
)
river2 = RiverModel(
    name="River2",
    x_coordinate=2,
    y_coordinate=2,
    mean_discharge=us.UnitfulDischarge(value=200, units=us.UnitTypesDischarge.cms),
    # Mean discharge is used create default discharge values for the river, you can still overwrite it later.
)

discharge1 = f.DischargeConstant(
    river=river1,
    discharge=us.UnitfulDischarge(value=100, units=us.UnitTypesDischarge.cms)
)
discharge2 = f.DischargeConstant(
    river=river2,
    discharge=us.UnitfulDischarge(value=200, units=us.UnitTypesDischarge.cms)
)

# Inspect
df1 = discharge1.to_dataframe(time_frame=time_frame)
df2 = discharge2.to_dataframe(time_frame=time_frame)

df1.plot(title="Constant Discharge River 1", xlabel="Time", ylabel="Discharge (cms)", legend=True, figsize=(5, 2))
df2.plot(title="Constant Discharge River 2", xlabel="Time", ylabel="Discharge (cms)", legend=True, figsize=(5, 2))


🧩 7. Combine Forcings and Create Synthetic Event

All defined forcings are collected into a single dictionary, which is used to construct a SyntheticEvent.

In [ ]:
# Create the forcings dictionary
# The keys of the dictionary are the forcing types, and the values are lists of the corresponding forcing objects.
forcings = {
    f.ForcingType.WATERLEVEL: [water_levels],
    f.ForcingType.RAINFALL: [rainfall],
    f.ForcingType.WIND: [wind],
    f.ForcingType.DISCHARGE: [discharge1, discharge2],
}

# Create a SyntheticEvent with the forcings and time frame
event = SyntheticEvent(
    name="example_synthetic_event",
    time=time_frame,
    forcings=forcings,
)

💾 8. Save the Event to a FloodAdapt Database

Finally, we save the event to a FloodAdapt database.

In [ ]:
# Save the event to the database
fa = FloodAdapt(database_path=Path("path/to/database")) # TODO
fa.save_event(event=event)